In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from pyhive import presto
from pymongo import MongoClient
import warnings
warnings.filterwarnings("ignore")
import my_func

presto_conn = presto.connect(
    host='bi-presto.serving.data.production.internal',
    port=80,
    protocol='http',
    catalog='hive',
    username='mayank.jha@rapido.bike',
    # requests_kwargs=req_kw,
)

In [6]:
start_date = '2021-11-08'
end_date = '2021-11-21'
city = 'Kolkata'

In [29]:
# Login hours

def get_login_hours(start_date, end_date, city):
    print("fetching Login hours for : ",start_date," to ", end_date)
    orders_query = """   SELECT 
            userid as rider, 
            CAST(WEEK(DATE(date_parse(yyyymmdd,'%Y%m%d'))) as int) as week,
            --date_format(date_parse(yyyymmdd, '%Y%m%d'), '%Y-%m-%d') AS logindate, 
            cast(sum(duration) as double)/cast((60*60*1000) as double) as login_hours,
            SUM(CASE WHEN quarter_hour >= '0600' and quarter_hour <= '1159' then DURATION ELSE 0 END)/cast((60*60*1000) as double) AS MORNING_DURATION,
            SUM(CASE WHEN quarter_hour >= '1200' and quarter_hour <= '1759' then DURATION ELSE 0 END)/cast((60*60*1000) as double) AS AFTERNOON_DURATION,
            SUM(CASE WHEN quarter_hour >= '1800' and quarter_hour <= '2359' then DURATION ELSE 0 END)/cast((60*60*1000) as double) AS EVENING_DURATION
        FROM hive.datasets.captain_login_hours
        WHERE yyyymmdd >= '{sd}' AND yyyymmdd <= '{ed}'
        AND status in ('2','3','6','7','8','10')
        AND userid in (SELECT captainId from datasets.captain_single_view WHERE (lower(registeredcity) = lower('{ct}') or lower(lastridecity) = lower('{ct}'))  AND activationdate is not null)
        GROUP BY 1,2          
               """.format(sd = start_date.replace('-',''), ed = end_date.replace('-',''), ct = city)
    #print(orders_query)
    df_lh = pd.read_sql(orders_query, presto_conn)
    
    print(" fetched Login hours", len(df_lh))

    return df_lh

df_login_hours = pd.DataFrame()

date_list = [pd.to_datetime(start_date) + timedelta(days=x) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days + 1)
        if (pd.to_datetime(start_date) + timedelta(days=x)).weekday() == 0]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]

for day in date_list : 
    login_hours = get_login_hours(day, (pd.to_datetime(day) + timedelta(6)).strftime('%Y-%m-%d'), city)
    x = pd.to_datetime(day).strftime('%Y-%V')
    #login_hours['week'] = x[-2:]
    #login_hours['week'] = login_hours['week'].apply(pd.to_numeric)
    df_login_hours = pd.concat([df_login_hours,login_hours])

df_login_hours.head()

fetching Login hours for :  2021-11-08  to  2021-11-14
 fetched Login hours 16736
fetching Login hours for :  2021-11-15  to  2021-11-21
 fetched Login hours 17636


,rider,week,login_hours,MORNING_DURATION,AFTERNOON_DURATION,EVENING_DURATION
0,5ddb9bb9a80df9312a193178,45,10.595000,0.220278,6.000000,4.374722
1,5d779557d0286d106d83fb15,45,0.191111,0.000000,0.000000,0.191111
2,5c9b024ab50ab0456be569c9,45,0.931944,0.892222,0.015556,0.024167
3,5ccfb5ff3d65ca5e256b072a,45,2.784722,2.001667,0.783056,0.000000
4,5e4bf4a1ed9c2b1ac1062870,45,1.801389,0.000000,0.971111,0.830278


In [30]:
# Earnings

def get_earnings():

    orders_query = """   
    select riderid, 
        week,
        sum(amount) as orders_earnings
    from
        (select distinct riderId as riderid,
                orderid,
                CAST(WEEK(DATE(date_parse(yyyymmdd,'%Y%m%d'))) as int) as week,
                cast(totalearning as double) as amount 
            from raw.mongodb_rapidopayroll_riderspaymentnew_immutable
            where yyyymmdd between '{sd}' and '{ed}'
                and city = '{ct}'
                and transactionType = 'orders' 
                and status = 'success'
        )
    group by 1,2
               """.format(sd = start_date.replace('-',''), ed = end_date.replace('-',''), ct = city)
    
    #print(orders_query)
    df_orders_data = pd.read_sql(orders_query, presto_conn)
    return df_orders_data

df_earnings = get_earnings()
df_earnings.head()

,riderid,week,orders_earnings
0,6059fcd1e757b2ac686e7161,45,254.952
1,61499ec3de864c4598030395,45,1908.792
2,617e8d16c324824411e3ca6f,45,640.000
3,5eccd468dcc41b6b2c32554e,46,620.968
4,60b34765270b9c59c1dd9a72,46,1156.616


In [31]:
# Find the incentive eligible and incentive achieved

def get_incentive_eligible(start_date, end_date, city):
    
    print("fetching eligible incentive for : ",start_date," to ", end_date)

    incentive_eligible = """with incentive as 
            (select distinct _id as incentive_id
                from hive.raw.mongodb_rapidopayroll_incentives_immutable
                where json_array_contains(cities, '{ct}')
                    and startDate between '{sd}' and '{ed}'
                    and endDate between '{sd}' and '{ed}'
            ),
daily_incentive as
            (select riderid,
                    --yyyymmdd,
                    sum(daily_incentive_achieved) as daily_incentive
                from
                    (select distinct riderid,
                            yyyymmdd,
                            tincentiveIdl,
                            subIncentiveId,
                            incentivestage,
                            cast(amount as double) as daily_incentive_achieved
                        from raw.mongodb_rapidopayroll_riderspaymentnew_immutable
                        where incentivetype = 'Daily' 
                            and status = 'success' 
                            and yyyymmdd between '{sd1}' and '{ed1}'
                            and city = '{ct}'
                    ) x
                    join incentive as i
                        on x.tincentiveIdl = i.incentive_id
                group by 1
            ),
weekly_incentive as
            (select riderid,
                    --yyyymmdd,
                    sum(weekly_incentive_achieved) as weekly_incentive
                from
                    (select distinct riderid,
                            yyyymmdd,
                            tincentiveIdl,
                            subIncentiveId,
                            incentivestage,
                            cast(amount as double) as weekly_incentive_achieved
                        from raw.mongodb_rapidopayroll_riderspaymentnew_immutable
                        where incentivetype = 'Weekly Fixed' 
                            and status = 'success' 
                            and yyyymmdd between '{sd1}' and '{ed1}'
                            and city = '{ct}'
                    ) x
                    join incentive as i
                        on x.tincentiveIdl = i.incentive_id
                group by 1
            )
            select coalesce(d.riderid, w.riderid) as riderid,
                --coalesce(d.yyyymmdd, w.yyyymmdd) as yyyymmdd,
                d.daily_incentive,
                w.weekly_incentive
            from daily_incentive as d
                full outer join 
                    weekly_incentive as w
                        on d.riderid = w.riderid --and d.yyyymmdd = w.yyyymmdd                  
               """.format(sd = start_date, ed = end_date, sd1 = start_date.replace('-',''), ed1 = end_date.replace('-',''),ct = city)
    
    #print(incentive_eligible)

    df_incentive_eligible = pd.read_sql(incentive_eligible, presto_conn)
    
    print(" fetching incentive eligible captains", len(df_incentive_eligible))

    return df_incentive_eligible

df_incentive_eligible = pd.DataFrame()

date_list = [pd.to_datetime(start_date) + timedelta(days=x) for x in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days + 1)
        if (pd.to_datetime(start_date) + timedelta(days=x)).weekday() == 0]
date_list = [x.strftime('%Y-%m-%d') for x in date_list]

for day in date_list : 
    incentive_eligible = get_incentive_eligible(day, (pd.to_datetime(day) + timedelta(7)).strftime('%Y-%m-%d'), city)
    x = pd.to_datetime(day).strftime('%Y-%V')
    incentive_eligible['week'] = x[-2:]
    incentive_eligible['week'] = incentive_eligible['week'].apply(pd.to_numeric)
    df_incentive_eligible = pd.concat([df_incentive_eligible,incentive_eligible])

df_incentive_eligible.head()

fetching eligible incentive for :  2021-11-08  to  2021-11-15
 fetching incentive eligible captains 4299
fetching eligible incentive for :  2021-11-15  to  2021-11-22
 fetching incentive eligible captains 5117


,riderid,daily_incentive,weekly_incentive,week
0,5f34ac010fbf96a40a50f833,160.0,NaN,45
1,5e198784013a515f249260d9,20.0,NaN,45
2,5cb38a8654bc7263ff39641d,300.0,NaN,45
3,5f51c4118172472de19f9d22,5.0,NaN,45
4,5cda7031d2ee6e6cf2e84a8f,15.0,NaN,45


In [32]:
# Rides & Ratings

rides_query = """SELECT rider,
                        --orderdate,
                        CAST(WEEK(DATE(date_parse(orderdate,'%Y-%m-%d'))) as int) as week,
                        sum(case when serviceobj_service = 'Link' then 1 else 0 end) as link_orders,
                        sum(case when serviceobj_service = 'Delivery' then 1 else 0 end) as delivery_orders,
                        sum(case when serviceobj_service not in ('Link','Delivery') then 1 else 0 end) as other_orders,
                        count(distinct orderdate) as active_days
                    from legacy.orders
                    where orderdate >= '{sd}'
                        and orderdate <= '{ed}'
                        and status = 'dropped'
                        and spdfraud_flag != 1
                        and serviceobj_city = '{ct}'
                    group by 1,2""".format(sd=start_date, ed= end_date, ct = city)

df_rides = pd.read_sql(rides_query, presto_conn)
df_rides.head()

,rider,week,link_orders,delivery_orders,other_orders,active_days
0,61722c444c6ba176ecdb7f32,45,20,0,0,5
1,617961b5fbce667efaa3eccb,45,18,0,0,3
2,5cefb35f377155163c0ae215,45,3,1,0,2
3,5ce912a625ee3218d4df17bc,45,22,0,0,6
4,5cb6d5b154bc7263ff3defe8,45,11,0,0,2


In [33]:
df_rides.rename(columns={'rider':'captain_id'}, inplace=True)
# df_rides = df_rides.groupby(['captain_id','week']).agg({'link_orders':'mean'}).reset_index()
# df_rides.head()

In [34]:
df_login_hours.rename(columns={'rider':'captain_id'}, inplace=True)
# df_login_hours = df_login_hours.groupby(['captain_id','week']).agg({'login_hours':'mean'}).reset_index()
# df_login_hours.head()

In [35]:
df_incentive_eligible.rename(columns={'riderid':'captain_id'}, inplace=True)
# df_incentive_eligible = df_incentive_eligible.groupby(['captain_id','week']).agg({'daily_incentive':'mean'}).reset_index()
# df_incentive_eligible.head()

In [39]:
df_test = pd.read_csv('Kolkata_subscription_test.csv')
df_test['tag'] = 'Test'
df_control = pd.read_csv('Kolkata_subscription_control.csv')
df_control['tag'] = 'Control'

In [40]:
df_captains = pd.concat([df_test, df_control])

In [21]:
# Rides & Ratings

rides_query = """select distinct userid as captain_id, 1 as subscribed from experiments.captain_subscription where status = 'SUCCESS' and date(starttime) = date('2021-11-14')  and city = '5bc5acb112477c2ece769599'"""

df_subscription = pd.read_sql(rides_query, presto_conn)
df_subscription.head()

,captain_id,subscribed
0,5ee81fe3a61758dbc5206089,1
1,5d20c0fd3b752c45cf9a77d5,1
2,5c80f3a25e042733c9c4ee4d,1
3,5cfa938ac0b26018f5dca7a9,1
4,5cc460583d65ca5e25648cf1,1


In [41]:
df_captains = df_captains.merge(df_subscription, how = 'left', on = 'captain_id')

In [49]:
df_captains = df_captains.merge(df_rides, how = 'left', on = 'captain_id')
# df_captains = df_captains.merge(df_earnings, how = 'left', on = ['captain_id','week'])
df_captains = df_captains.merge(df_login_hours, how = 'left', on = ['captain_id','week'])
df_captains = df_captains.merge(df_incentive_eligible, how = 'left', on = ['captain_id','week'])

In [50]:
df_captains.head()

,captain_id,mobilenumber,segment,tag,subscribed,week,link_orders,delivery_orders,other_orders,active_days,login_hours,MORNING_DURATION,AFTERNOON_DURATION,EVENING_DURATION,daily_incentive,weekly_incentive
0,5d0bbf64c4021c473c923faf,8336849036,LP_D_HO,Test,NaN,45.0,14.0,5.0,2.0,4.0,24.087222,3.788333,9.625000,9.847778,65.0,NaN
1,5d0bbf64c4021c473c923faf,8336849036,LP_D_HO,Test,NaN,46.0,12.0,9.0,4.0,6.0,34.491389,4.266389,11.195556,18.871111,100.0,NaN
2,5c4c14804a267149c77bbe11,8100408166,LP_D_UHO,Test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5e363e85ca251b6a5354c7d9,7980043234,LP_Intra_HO,Test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,60af735fc41d748a3d701fed,7044867626,MP_D_HO,Test,NaN,45.0,5.0,2.0,4.0,2.0,10.401389,0.000000,3.077778,7.323611,60.0,NaN


In [51]:
adoption = pd.DataFrame(columns=['segment','test','control'])
for s in df_captains['segment'].unique():
    num_test = df_captains[(df_captains['segment']==s) & (df_captains['tag']=='Test') & (df_captains['subscribed']==1)]['captain_id'].nunique()
    num_control = df_captains[(df_captains['segment']==s) & (df_captains['tag']=='Control') & (df_captains['subscribed']==1)]['captain_id'].nunique()
    den_test = df_captains[(df_captains['segment']==s) & (df_captains['tag']=='Test')]['captain_id'].nunique()
    den_control = df_captains[(df_captains['segment']==s) & (df_captains['tag']=='Control')]['captain_id'].nunique()
    adoption_test = round(num_test/den_test*100.0,0) if den_test != 0 else 0
    adoption_control = round(num_control/den_control*100.0,0) if den_control != 0 else 0
    x = pd.DataFrame({"segment" : [s], "test" : [adoption_test], "control" : [adoption_control]})
    adoption = pd.concat([adoption, x])
adoption

,segment,test,control
0,LP_D_HO,12.0,4.0
0,LP_D_UHO,12.0,4.0
0,LP_Intra_HO,0.0,1.0
0,MP_D_HO,15.0,8.0
0,LP_D_LO,6.0,2.0
0,MP_D_UHO,20.0,7.0
0,LP_D_MO,11.0,4.0
0,LP_Intra_LO,0.0,1.0
0,LP_Inter_MO,0.0,0.0
0,LP_Inter_LO,0.0,0.0


In [94]:
df_captains[~df_captains['segment'].isin(['HP_D_HO','HP_D_UHO','UHP_D_UHO','ZP'])].pivot_table(index='segment', columns=['tag'], values = 'captain_id', aggfunc = 'nunique')

tag,Control,Test
segment,,
LP_D_HO,482,468
LP_D_LO,504,500
LP_D_MO,574,633
LP_D_UHO,559,547
LP_Inter_HO,149,141
LP_Inter_LO,541,581
LP_Inter_MO,237,253
LP_Inter_UHO,149,148
LP_Intra_HO,274,290


In [59]:
df_captains[~df_captains['segment'].isin(['HP_D_HO','HP_D_UHO','UHP_D_UHO','ZP']) & (df_captains['subscribed']==1)].pivot_table(index='segment', columns=['tag','week'], values = 'captain_id', aggfunc = 'nunique')


tag          Control        Test      
week            45.0  46.0  45.0  46.0
segment                               
LP_D_HO         18.0  16.0  48.0  50.0
LP_D_LO          8.0   8.0  25.0  22.0
LP_D_MO         22.0  20.0  60.0  59.0
LP_D_UHO        22.0  22.0  57.0  59.0
LP_Inter_HO      1.0   NaN   NaN   NaN
LP_Inter_LO      2.0   1.0   NaN   NaN
LP_Inter_UHO     3.0   2.0   NaN   NaN
LP_Intra_HO      2.0   1.0   NaN   NaN
LP_Intra_LO      4.0   4.0   NaN   NaN
LP_Intra_MO      5.0   5.0   NaN   NaN
LP_Intra_UHO     3.0   3.0   NaN   NaN
MP_D_HO         30.0  26.0  57.0  49.0
MP_D_LO          1.0   1.0   1.0   NaN
MP_D_MO         17.0  16.0  41.0  36.0
MP_D_UHO        37.0  35.0  96.0  90.0

In [58]:
df_captains[~df_captains['segment'].isin(['HP_D_HO','HP_D_UHO','UHP_D_UHO','ZP']) & (df_captains['subscribed']==1)].pivot_table(index='segment', columns=['tag','week'], values = 'link_orders', aggfunc = 'mean')


tag             Control                  Test           
week               45.0       46.0       45.0       46.0
segment                                                 
LP_D_HO       21.000000  15.312500  14.041667  14.440000
LP_D_LO       26.875000  12.250000   8.640000  11.727273
LP_D_MO       25.590909  17.700000  12.350000  16.661017
LP_D_UHO      25.545455  28.000000  14.421053  22.830508
LP_Inter_HO    0.000000        NaN        NaN        NaN
LP_Inter_LO    8.500000  52.000000        NaN        NaN
LP_Inter_UHO   7.666667   5.000000        NaN        NaN
LP_Intra_HO    3.500000  11.000000        NaN        NaN
LP_Intra_LO    2.250000  10.500000        NaN        NaN
LP_Intra_MO    5.400000   4.800000        NaN        NaN
LP_Intra_UHO   6.000000   4.666667        NaN        NaN
MP_D_HO       26.633333  24.307692  25.824561  21.142857
MP_D_LO       19.000000  31.000000   2.000000        NaN
MP_D_MO       32.117647  20.250000  19.121951  20.000000
MP_D_UHO      34.513514  27.171429  28.375000  26.266667

In [60]:
df_captains[~df_captains['segment'].isin(['HP_D_HO','HP_D_UHO','UHP_D_UHO','ZP']) & (df_captains['subscribed']==1)].pivot_table(index='tag', columns=['week'], values = 'link_orders', aggfunc = 'mean')


week,45.0,46.0
tag,,
Control,25.437500,21.285714
Test,19.311688,20.356164


In [53]:
df_captains['total_orders'] = df_captains['link_orders'] + df_captains['delivery_orders'] + df_captains['other_orders']

In [54]:
df_captains[~df_captains['segment'].isin(['HP_D_HO','HP_D_UHO','UHP_D_UHO','ZP']) & (df_captains['subscribed']==1)].pivot_table(index='segment', columns=['tag','week'], values = 'total_orders', aggfunc = 'mean')


tag             Control                  Test           
week               45.0       46.0       45.0       46.0
segment                                                 
LP_D_HO       28.722222  20.687500  20.312500  20.220000
LP_D_LO       28.000000  14.250000  13.800000  16.363636
LP_D_MO       28.181818  19.600000  19.483333  21.491525
LP_D_UHO      30.000000  32.045455  23.280702  29.491525
LP_Inter_HO    3.000000        NaN        NaN        NaN
LP_Inter_LO    8.500000  52.000000        NaN        NaN
LP_Inter_UHO  23.333333  14.500000        NaN        NaN
LP_Intra_HO    8.000000  18.000000        NaN        NaN
LP_Intra_LO   12.750000  17.000000        NaN        NaN
LP_Intra_MO   14.000000   5.200000        NaN        NaN
LP_Intra_UHO   6.000000   5.000000        NaN        NaN
MP_D_HO       28.600000  25.384615  29.315789  24.673469
MP_D_LO       19.000000  31.000000   5.000000        NaN
MP_D_MO       34.117647  21.125000  22.365854  21.944444
MP_D_UHO      36.486486  28.257143  34.156250  29.811111

In [56]:
df_captains[(df_captains['segment']=='MP_D_HO') & (df_captains['subscribed']==1)]

,captain_id,mobilenumber,segment,tag,subscribed,week,link_orders,delivery_orders,other_orders,active_days,login_hours,MORNING_DURATION,AFTERNOON_DURATION,EVENING_DURATION,daily_incentive,weekly_incentive,total_orders
94,61580c017fefc5988c86ee5a,8100648464,MP_D_HO,Test,1.0,45.0,19.0,0.0,0.0,5.0,14.986944,14.904722,0.082222,0.000000,NaN,NaN,19.0
95,61580c017fefc5988c86ee5a,8100648464,MP_D_HO,Test,1.0,46.0,19.0,0.0,0.0,4.0,13.548056,13.541667,0.000000,0.006389,NaN,NaN,19.0
348,5fa1472cff48feaf46f5b7be,6294844234,MP_D_HO,Test,1.0,45.0,26.0,0.0,1.0,4.0,25.090278,0.816944,8.264722,16.008611,330.0,NaN,27.0
349,5fa1472cff48feaf46f5b7be,6294844234,MP_D_HO,Test,1.0,46.0,10.0,0.0,0.0,3.0,14.744722,0.000000,5.617500,9.127222,60.0,NaN,10.0
430,5f8716fc506e13eb6d48eb96,7537878032,MP_D_HO,Test,1.0,46.0,19.0,0.0,0.0,6.0,13.060000,6.841111,1.770833,4.448056,60.0,NaN,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16667,611cc282f3feb181f303a6be,9831208773,MP_D_HO,Control,1.0,46.0,37.0,0.0,0.0,7.0,33.337222,19.242222,10.730278,3.364722,350.0,NaN,37.0
16671,5fbe469839593f42dc65cc12,7980559091,MP_D_HO,Control,1.0,46.0,38.0,0.0,0.0,6.0,29.479167,7.616389,20.296111,1.566667,390.0,NaN,38.0
16672,5fbe469839593f42dc65cc12,7980559091,MP_D_HO,Control,1.0,45.0,18.0,0.0,0.0,3.0,12.463889,5.001111,6.791667,0.671111,200.0,NaN,18.0
16721,60fb9edba0ea0de212733024,9674707401,MP_D_HO,Control,1.0,45.0,1.0,0.0,0.0,1.0,4.163889,0.000000,0.665000,3.498889,NaN,NaN,1.0
